## All Stats Combined
Here's the idea
- This notebook is made to be able to run all of our statistics very quickly (well not quickly but at least with one button push rather than hundreds) across all sample sizes, and runs within that
- End goal of this notebook is to get a table where all 6 statistics are computed in one output per one sample size - for a more detailed look into each of the statistics and what goes into them you can look into the individual notebooks
- This is going to be fairly nasty looking but we did this so we can press one button and get all the statistics all at once

In [19]:
import pandas as pd
import networkx as nx # documentation at https://networkx.org/documentation/
import matplotlib.pyplot as plt
from scipy import stats
import collections
import numpy as np

In [20]:
data = pd.read_csv("web-Stanford.txt", sep="\t")
in_degree_OG = data.groupby("ToNode").count()["FromNode"] #for in degree statistic
out_degree_OG = data.groupby("FromNode").count()["ToNode"] #for out degree statistic
G_Original = nx.DiGraph()
G_Original = nx.from_pandas_edgelist(data, "FromNode", "ToNode", create_using=nx.DiGraph())
wcc_OG = nx.weakly_connected_components(G_Original)
wcc_sizes_OG = collections.Counter([len(wcc) for wcc in wcc_OG]).keys()
scc_OG = nx.strongly_connected_components(G_Original)
scc_sizes_OG = collections.Counter([len(wcc) for wcc in scc_OG]).keys()
cc_OG = nx.average_clustering(G_Original)

In [22]:
list_of_nodes = np.unique(np.array(data["FromNode"]))
edge_holder = {}
for nodes in list_of_nodes:
    outlinks = list(data[data["FromNode"] == nodes]["ToNode"])
    edge_holder[nodes] = set(outlinks)

NameError: name 'counting' is not defined

In [24]:
hop_plot_3_OG = counting(3)

In [26]:
largestcc = max(nx.weakly_connected_components(G_Original), key=len)
edge_holder = {}
for nodes in list_of_nodes:
    if(nodes not in largestcc):
        continue
    outlinks = list(data[data["FromNode"] == nodes]["ToNode"])
    filtered = list(filter(lambda x: x in largestcc, outlinks))
    if(len(filtered) > 0):
        edge_holder[nodes] = set(filtered)

hop_plot_3_wcc_OG = counting(3)

In [11]:
def indegree():
    in_degree_sample = input_data.groupby("ToNode").count()["FromNode"]
    return stats.ks_2samp(in_degree_OG, in_degree_sample)[0] #returns the d-statistic

In [12]:
def outdegree():
    out_degree_sample = input_data.groupby("FromNode").count()["ToNode"]
    return stats.ks_2samp(out_degree_OG, out_degree_sample)[0] #returns the d-statistic

In [14]:
def WCC():
    G_sample = nx.DiGraph()
    G_sample = nx.from_pandas_edgelist(input_data, "FromNode", "ToNode", create_using=nx.DiGraph())
    wcc_sample = nx.weakly_connected_components(G_sample)
    wcc_sizes_sample = collections.Counter([len(wcc) for wcc in wcc_sample]).keys()
    return stats.ks_2samp(list(wcc_sizes_OG), list(wcc_sizes_sample))[0]

In [59]:
def SCC():
    G_sample = nx.DiGraph()
    G_sample = nx.from_pandas_edgelist(input_data, "FromNode", "ToNode", create_using=nx.DiGraph())
    scc_sample = nx.strongly_connected_components(G_sample)
    scc_sizes_sample = collections.Counter([len(wcc) for wcc in scc_sample]).keys()
    return stats.ks_2samp(list(scc_sizes_OG), list(scc_sizes_sample))[0]

In [93]:
def clusteringCoef():
    G_sample = nx.DiGraph()
    G_sample = nx.from_pandas_edgelist(input_data, "FromNode", "ToNode", create_using=nx.DiGraph())
    cc_sample = nx.average_clustering(G_sample)
    return cc_sample

In [51]:
def hopping(h, edge_holder): #now this returns all edges that can be reached in some amount of hops
    #this works for two steps now need to make it work for 3,4, etc...
    #first step is for each h create a new "edge_holder" that holds the elements it's connected ot in h steps
    edge_holder2 = edge_holder.copy()
    for nodes in edge_holder:
        last_nodes = edge_holder[nodes]
        hops = h
        while(hops > 1):
            temp = set()
            for outlinks in last_nodes:
                if(outlinks in edge_holder):
                    temp = set.union(temp, edge_holder[outlinks])            
            edge_holder2[nodes] = set.union(temp, edge_holder2[nodes])
            hops = hops - 1
            last_nodes = temp
    
    
            
    return edge_holder2

def counting(num_steps, edge_holder): #what actually needs to be run for hop plot to run
    e = hopping(num_steps, edge_holder);
    pairs = 0;
    for nodes in e:
        for outlinks in e[nodes]:
            if(outlinks in e and nodes in e[outlinks]):
                pairs += 1;
    
    return int(pairs/2); #since we end up double counting

In [53]:
def edge_holder_normal():
    list_of_nodes = np.unique(np.array(input_data["FromNode"]))
    edge_holder = {}
    for nodes in list_of_nodes:
        outlinks = list(data[data["FromNode"] == nodes]["ToNode"])
        edge_holder[nodes] = set(outlinks)
    return edge_holder

In [82]:
def edge_holder_wcc():
    G_smp = nx.DiGraph()
    G_smp = nx.from_pandas_edgelist(input_data, "FromNode", "ToNode", create_using=nx.DiGraph())
    largestcc = max(nx.weakly_connected_components(G_smp), key=len)
    edge_holder = {}
    for nodes in list_of_nodes:
        if(nodes not in largestcc):
            continue
        outlinks = list(input_data[input_data["FromNode"] == nodes]["ToNode"])
        filtered = list(filter(lambda x: x in largestcc, outlinks))
        if(len(filtered) > 0):
            edge_holder[nodes] = set(filtered)
    return edge_holder

In [95]:
ssizes = ["5%", "10%", "15%", "20%"]
runnumbers = ["1", "2", "3", "4", "5"]
typeofsamples = ["FF", "RNE", "RW", "RN"]

In [75]:
dic = {"RW5%" : [], "RNE5%": [], "FF5%": [], "RN5%": [],
       "RW10%" : [], "RNE10%": [], "FF10%": [], "RN10%": [], 
      "RW15%" : [], "RNE15%": [], "FF15%": [], "RN15%": [], 
       "RW20%" : [], "RNE20%": [], "FF20%": [], "RN20%": [], 
      }

In [96]:
dic2 = {"RW5%" : [], "RNE5%": [], "FF5%": [], "RN5%": [],
       "RW10%" : [], "RNE10%": [], "FF10%": [], "RN10%": [], 
      "RW15%" : [], "RNE15%": [], "FF15%": [], "RN15%": [], 
       "RW20%" : [], "RNE20%": [], "FF20%": [], "RN20%": [], 
      }

In [97]:
for ssize in ssizes:
    for typeofsample in typeofsamples:
        keyname = typeofsample + ssize #keeping track of where to store things
        cc = []
        for runnumber in runnumbers:
            
            name = ssize + "/" + typeofsample + "/" + "Output" + typeofsample + ssize + runnumber + ".csv"
            input_data = pd.read_csv(name);
            
            cc.append(clusteringCoef())
    
    
        dic2[keyname].append(cc)
        
            
            
        print(dic2)
op = pd.DataFrame.from_dict(dic2)
op.to_csv("ClusterCoef?.csv")

{'RW5%': [], 'RNE5%': [], 'FF5%': [[0.17951994255729964, 0.16755313921183476, 0.1671714837845175, 0.17491917887766908, 0.16968546183015012]], 'RN5%': [], 'RW10%': [], 'RNE10%': [], 'FF10%': [], 'RN10%': [], 'RW15%': [], 'RNE15%': [], 'FF15%': [], 'RN15%': [], 'RW20%': [], 'RNE20%': [], 'FF20%': [], 'RN20%': []}
{'RW5%': [], 'RNE5%': [[0.00010648115283594804, 0.00015734431194397124, 0.0, 3.656928481234579e-05, 0.0]], 'FF5%': [[0.17951994255729964, 0.16755313921183476, 0.1671714837845175, 0.17491917887766908, 0.16968546183015012]], 'RN5%': [], 'RW10%': [], 'RNE10%': [], 'FF10%': [], 'RN10%': [], 'RW15%': [], 'RNE15%': [], 'FF15%': [], 'RN15%': [], 'RW20%': [], 'RNE20%': [], 'FF20%': [], 'RN20%': []}
{'RW5%': [[0.12453266149139548, 0.13039878711178204, 0.1288119325325727, 0.1227326534597094, 0.1237568040523961]], 'RNE5%': [[0.00010648115283594804, 0.00015734431194397124, 0.0, 3.656928481234579e-05, 0.0]], 'FF5%': [[0.17951994255729964, 0.16755313921183476, 0.1671714837845175, 0.1749191788

{'RW5%': [[0.12453266149139548, 0.13039878711178204, 0.1288119325325727, 0.1227326534597094, 0.1237568040523961]], 'RNE5%': [[0.00010648115283594804, 0.00015734431194397124, 0.0, 3.656928481234579e-05, 0.0]], 'FF5%': [[0.17951994255729964, 0.16755313921183476, 0.1671714837845175, 0.17491917887766908, 0.16968546183015012]], 'RN5%': [[0.12827252277745266, 0.14757554705933848, 0.10442785000860369, 0.10696939549531728, 0.11352576886597106]], 'RW10%': [[0.13831132658747894, 0.1382077660742894, 0.14223101692455142, 0.13721333115023626, 0.14197899252423765]], 'RNE10%': [[5.324057641797402e-05, 0.000210070262562167, 0.0, 0.00014197501213941818, 0.00017676458679267276]], 'FF10%': [[0.1852046646936887, 0.18352968018609833, 0.18106590722343477, 0.18478195482283802, 0.1786746215248763]], 'RN10%': [[0.1727939092326368, 0.1977976557216357, 0.18058297766952644, 0.1772954068805168, 0.18611311867161337]], 'RW15%': [[0.14460842214322117, 0.14478207177163976, 0.1475730263845146, 0.14506932314144882, 0.14

In [76]:
for ssize in ssizes:
    for typeofsample in typeofsamples:
        keyname = typeofsample + ssize #keeping track of where to store things
        ind = []
        outd = []
        wcc = []
        scc = []
        hopnormal = []
        hopwcc = []
        cc = []
        for runnumber in runnumbers:
            
            name = ssize + "/" + typeofsample + "/" + "Output" + typeofsample + ssize + runnumber + ".csv"
            input_data = pd.read_csv(name);
            
            ind.append(indegree())
            outd.append(outdegree())
            wcc.append(WCC())
            scc.append(SCC())
            eh = edge_holder_normal()
            hopnormal.append(counting(3, eh))
            eh2 = edge_holder_wcc()
            hopwcc.append(counting(3, eh))
            cc.append(clusteringCoef())
    
    
        dic[keyname].append(ind)
        dic[keyname].append(outd)
        dic[keyname].append(wcc)
        dic[keyname].append(scc)
        dic[keyname].append(hopnormal)
        dic[keyname].append(hopwcc)
        dic[keyname].append(cc)
            
            
        print(dic)
op = pd.DataFrame.from_dict(dic)
op.to_csv("Finally?.csv")

{'RW10%': [], 'RNE10%': [], 'FF10%': [[0.17299981719085217, 0.17926699883428404, 0.18281637059548228, 0.18755275794973475, 0.18801419265672376], [0.29941480664858056, 0.29156563405183966, 0.3107595732651127, 0.29691405553052286, 0.2955967465510163], [0.492, 0.992, 0.992, 0.992, 0.992], [0.5069320066334991, 0.532991452991453, 0.5053399668325042, 0.5110457516339869, 0.4973856209150327], [746586, 807638, 642824, 742127, 728667], [746586, 807638, 642824, 742127, 728667]], 'RN10%': [], 'RW15%': [], 'RNE15%': [], 'FF15%': [], 'RN15%': [], 'RW20%': [], 'RNE20%': [], 'FF20%': [], 'RN20%': []}
{'RW10%': [], 'RNE10%': [[0.6300847909744887, 0.6283808132417845, 0.6298065959231889, 0.6258077790452725, 0.6317014591859351], [0.8766029654588731, 0.875978906026148, 0.8753988278410889, 0.8750679529101901, 0.8769302743176025], [0.4232592592592593, 0.4283636363636364, 0.4054901960784314, 0.4410980392156863, 0.4398490566037736], [0.9866666666666667, 0.9866666666666667, 0.9911111111111112, 0.991111111111111

{'RW10%': [[0.40346870584310784, 0.3971984249718577, 0.4035518004244026, 0.4030193946602228, 0.39633421537350927], [0.5336433493472208, 0.5285305103518136, 0.5297092878372169, 0.5313239773916114, 0.5244904299058543], [0.38133333333333336, 0.308, 0.34410126582278483, 0.3084878048780488, 0.328], [0.4326726726726727, 0.4088888888888889, 0.4333986928104575, 0.4127777777777778, 0.41491628614916287], [810628, 840169, 828361, 822900, 838134], [810628, 840169, 828361, 822900, 838134]], 'RNE10%': [[0.6300847909744887, 0.6283808132417845, 0.6298065959231889, 0.6258077790452725, 0.6317014591859351], [0.8766029654588731, 0.875978906026148, 0.8753988278410889, 0.8750679529101901, 0.8769302743176025], [0.4232592592592593, 0.4283636363636364, 0.4054901960784314, 0.4410980392156863, 0.4398490566037736], [0.9866666666666667, 0.9866666666666667, 0.9911111111111112, 0.9911111111111112, 0.9911111111111112], [3083, 3132, 2979, 3122, 2904], [3083, 3132, 2979, 3122, 2904]], 'FF10%': [[0.17299981719085217, 0.

{'RW10%': [[0.40346870584310784, 0.3971984249718577, 0.4035518004244026, 0.4030193946602228, 0.39633421537350927], [0.5336433493472208, 0.5285305103518136, 0.5297092878372169, 0.5313239773916114, 0.5244904299058543], [0.38133333333333336, 0.308, 0.34410126582278483, 0.3084878048780488, 0.328], [0.4326726726726727, 0.4088888888888889, 0.4333986928104575, 0.4127777777777778, 0.41491628614916287], [810628, 840169, 828361, 822900, 838134], [810628, 840169, 828361, 822900, 838134]], 'RNE10%': [[0.6300847909744887, 0.6283808132417845, 0.6298065959231889, 0.6258077790452725, 0.6317014591859351], [0.8766029654588731, 0.875978906026148, 0.8753988278410889, 0.8750679529101901, 0.8769302743176025], [0.4232592592592593, 0.4283636363636364, 0.4054901960784314, 0.4410980392156863, 0.4398490566037736], [0.9866666666666667, 0.9866666666666667, 0.9911111111111112, 0.9911111111111112, 0.9911111111111112], [3083, 3132, 2979, 3122, 2904], [3083, 3132, 2979, 3122, 2904]], 'FF10%': [[0.17299981719085217, 0.

{'RW10%': [[0.40346870584310784, 0.3971984249718577, 0.4035518004244026, 0.4030193946602228, 0.39633421537350927], [0.5336433493472208, 0.5285305103518136, 0.5297092878372169, 0.5313239773916114, 0.5244904299058543], [0.38133333333333336, 0.308, 0.34410126582278483, 0.3084878048780488, 0.328], [0.4326726726726727, 0.4088888888888889, 0.4333986928104575, 0.4127777777777778, 0.41491628614916287], [810628, 840169, 828361, 822900, 838134], [810628, 840169, 828361, 822900, 838134]], 'RNE10%': [[0.6300847909744887, 0.6283808132417845, 0.6298065959231889, 0.6258077790452725, 0.6317014591859351], [0.8766029654588731, 0.875978906026148, 0.8753988278410889, 0.8750679529101901, 0.8769302743176025], [0.4232592592592593, 0.4283636363636364, 0.4054901960784314, 0.4410980392156863, 0.4398490566037736], [0.9866666666666667, 0.9866666666666667, 0.9911111111111112, 0.9911111111111112, 0.9911111111111112], [3083, 3132, 2979, 3122, 2904], [3083, 3132, 2979, 3122, 2904]], 'FF10%': [[0.17299981719085217, 0.

## End